# Wie Maschinen menschliche Sprache lernen: Word Embeddings 🧑‍💻

Heute trainieren wir ein sog. *Sprachmodell*, also ein künstliches Modell natürlicher Sprache. Dazu "füttern" wir einen darauf spezialisierten Algorithmus mit einer riesigen Menge an Sprachbeispielen. Als Sprachbeispiele verwenden wir einen Datensatz von [Wortschatz Leipzig](https://wortschatz.uni-leipzig.de/de) mit 100.000 Sätzen aus deutschen Zeitungsartikeln aus dem Jahr 2022. 

Grob formuliert, schaut sich der Algorithmus die Beispiele immer und immer wieder an und analysiert, wie und wo die einzelnen Wörter darin auftreten. Über die große Anzahl an Beispielen hinweg, findet der Algorithmus so Beziehungen zwischen den Wörtern sowie typische Muster, innerhalb derer sie auftreten (z.B. das vor "bin" typischerweise "ich" steht). Dadurch lernt der Algorithmus im Idealfall annäherungsweise die Bedeutung, die die Wörter für unser menschliches Sprachverständnis haben. 

Als Erstes importieren wir Python-Module, die wir zum Training benötigen und nehmen auch ein paar Einstellungen vor. Führe diese Zelle aus, indem Du in sie reinklickst und anschließend `Shift + Enter` drückt. Diesen Befehl sollst Du ab sofort bei jeder Code-Zelle ausführen.

In [1]:
import gensim
import pandas as pd
import logging
import warnings
from tqdm import tqdm

warnings.simplefilter(action="ignore", category=FutureWarning)

logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

## 1. Daten einlesen

Nun wollen wir unseren Datensatz "deu_news_2022_100K-sentences.txt" aus dem Ordner "data" einlesen. Wenn Du die Datei mit dem Standardprogramm auf Deinem Rechner öffnest, siehst Du, dass es sich um eine Art Tabelle handelt, mit je einer Zahl sowie einem vollständigen Satz pro Zeile. 

Der Algorithmus, den wir zum Trainieren unseres Sprachmodells verwenden, nämlich [*word2vec*](https://en.wikipedia.org/wiki/Word2vec), verlangt eine Liste mit Sätzen, wobei jeder Satz wiederum als Liste mit Wörtern erwartet wird. Deshalb tokenisieren wir jeden Satz beim Einlesen und hängen ihn als Wortliste der Satzliste an. Zusätzlich entfernen wir die Zahl am Zeilenanfang. Führe die Zelle wiederum mittels `Shift + Enter` aus.

In [2]:
data = "data_100K/data/deu_news_2022_100K-sentences.txt"

with open(data, encoding="utf-8") as f:
    
    #Tokenisieren inkl. Wegsplitten der Indizes am Zeilenanfang
    sentences = [sentence.split()[1:] for sentence in tqdm(f)]

100000it [00:00, 223165.85it/s]


Inspizieren wir mal die eingelesenen Daten und lassen uns zwei Sätze ausgeben. Eckige Klammern begrenzen bei Python Listen, wobei die einzelnen Elemente einer Liste jeweils mit Kommata voneinander abgetrennt sind.

In [3]:
print(sentences[10:12])

[['"61', 'Gegentore', 'wie', 'in', 'der', 'vergangenen', 'Saison', '-', 'das', 'ist', 'eigentlich', 'abstiegsreif.'], ['"80', 'Prozent', 'der', 'Landschaften', 'sind', 'in', 'einem', 'ökologisch', 'schlechten', 'Zustand",', 'mahnt', 'Wolfgang', 'Lucht,', 'Professor', 'am', 'Potsdam-Institut', 'für', 'Klimafolgenforschung.']]


Das sieht schon mal sehr gut aus!

Allerdings sehen wir, dass die Wörter diverse Zeichen enthalten, die wir beim Training nicht gebrauchen können. Das Anführungzeichen hinter 'Zustand"' etwa hilft dem Algorithmus nicht, sich der Bedeutung von "Zustand" anzunähern. Ebenso unnötig ist der "." am Ende von "Klimafolgenforschung.". Wir müssen unsere Daten also bereinigen. Diesen Schritt nennt man *Preprocessing*.

## 2. Daten bereinigen (Preprocessing)

Die folgende Zelle definiert die Funktion `strip_special_signs`, die wir in der Zelle darunter auf unsere Daten anwenden. Sie identifiziert in einem ersten Schritt induktiv sämtliche nicht-alphanumerischen Zeichen in unseren Daten und entfernt diese im zweiten Schritt von sämtlichen Wortanfängen und -enden. Den Code musst Du nicht im Detail nachvollziehen können. Wenn er Dich aber interessiert, kannst Du die Kommentare (alles was mit `#` beginnt) lesen. Sie beschreiben jeweils, was der Code davor bzw. darunter tut. Führe die beiden Zellen in jedem Fall aus.

In [4]:
def strip_special_signs(list_of_lists):
    
    #Schritt 1
    special_signs = set() #Definieren eines noch leeren Sets (math. Menge), an das wir sämtliche Spezialzeichen (Zeichen, die nicht zu den normal_signs gehören), anhängen
    normal_signs = list("abcdefghijklmnopqrstuvwxyzäöüß1234567890") #Schaffen einer Liste mit sämtlichen normalen alphanumerischen Zeichen
    
    print("Identifying special signs") #Ausgabe des aktuellen Schritts
    
    #Iteration über alle Sätze...
    for sentence in tqdm(sentences):
        #...und Wörter in unseren Daten
        for word in sentence:
            #Überprüfen, ob das erste Zeichen (mit Index 0) beim jeweiligen Wort in der Liste normal_signs ist und wenn NICHT...
            if word[0].lower() not in normal_signs:
                #...Anhängen des jeweiligen Zeichens an das Set special_signs
                special_signs.add(word[0])
            #ebenfalls Überprüfen, ob letztes Zeichen beim jeweiligen Wort in der Liste normal_signs ist und wenn NICHT...
            if word[-1].lower() not in normal_signs:  
                #Anhängen des jeweiligen Zeichens an das Set special_signs
                special_signs.add(word[-1])
        
    #Schritt 2
    print("Stripping off special signs") #Ausgabe des aktuellen Schritts
    
    preprocessed_sentences = [] #Definieren einer noch leeren Liste, an die wir sämtliche bereingten Sätze anhängen
    
    #Iteration über alle noch unbereingten Sätze...
    for sentence in tqdm(sentences):
        preprocessed_sentence = [] #Definieren einer leeren Liste, an die wir sämtliche bereingten Wörter EINES Satzes anhängen (Liste wird bei jeder Iteration neu geschaffen)
        #...und unbereingten Wörter
        for word in sentence:
            #Entfernen (strip) aller Spezialzeichen am Wortanfang und -ende
            preprocessed_word = word.strip("".join(special_signs))
            #Sofern Wort länger als null Buchstaben...
            if len(preprocessed_word) > 0:
                #Anhängen an Liste mit sämtlichen bereinigten Wörtern
                preprocessed_sentence.append(preprocessed_word)
        #Anhängen der Liste mit bereingten Wörtern an Liste mit bereinigten Sätzen
        preprocessed_sentences.append(preprocessed_sentence)
        
    #Rückgabe der Liste mit bereingten Sätzen
    return preprocessed_sentences

In [5]:
preprocessed_sentences = strip_special_signs(sentences)

 19%|██████▎                          | 19226/100000 [00:00<00:00, 92863.76it/s]

Identifying special signs


  5%|█▌                                | 4664/100000 [00:00<00:04, 23658.13it/s]

Stripping off special signs


100%|████████████████████████████████| 100000/100000 [00:04<00:00, 23257.02it/s]


`preprocessed_sentences` bezeichnet jetzt die Liste mit bereinigten Sätzen. Schauen wir uns nochmal die gleichen Sätze wie oben an:

In [6]:
print(preprocessed_sentences[10:12])

[['61', 'Gegentore', 'wie', 'in', 'der', 'vergangenen', 'Saison', 'das', 'ist', 'eigentlich', 'abstiegsreif'], ['80', 'Prozent', 'der', 'Landschaften', 'sind', 'in', 'einem', 'ökologisch', 'schlechten', 'Zustand', 'mahnt', 'Wolfgang', 'Lucht', 'Professor', 'am', 'Potsdam-Institut', 'für', 'Klimafolgenforschung']]


Das hat doch wunderbar geklappt. 

Nun sind unsere Daten bereit fürs Training!

## 3. Training

Während die Bedeutung von Wörtern bei Menschen im Sprachzentrum des Gehirns abgespeichert ist, so werden Wortbedeutungen bei word2vec in Form von Vektoren repräsentiert, also grob formuliert als Zahlenreihen, z.B. so: 

    [0.9823969, -0.16720027,  0.69778556, -0.10027876,  0.70647484,  1.0204794, ...].

### 3.1. Was ist ein Vektor?

Ein Vektor besteht aus einer bestimmten Anzahl an Zahlen, wobei jede Zahl angibt, inwiefern ein bestimmtes *Feature* bei einem gegebenen Wort zutrifft. Vereinfacht kann man sich einen Vektor als eine Art Reihe an numerischen Antworten auf sinnvolle Fragen vorstellen. Die erste Zahl im Vektor (das erste Feature) stünde z.B. immer für die Anzahl an Buchstaben in einem gegebenen Wort, die zweite Zahl für die Auftretenshäufigkeit im Satz, etc. Ein komplexer Algorithmus wie word2vec kodiert allerdings keine für Menschen sinnvollen Features (Frage-Antwort-Paare), sondern die abstrakten Beziehungen und Muster zwischen Wörtern, die er beim Training entdeckt. Bei genügend Daten kann ein Modell so durchaus semantisch-syntaktisch sinnvolle Repräsentationen von Wörtern erlernen. Insgesamt entsteht beim Training ein Vektorraum, in dem die einzelnen Wortvektoren eingebettet sind, weswegen wir auch von *Word Embeddings* sprechen.

### 3.2. Wie funktioniert das Training?

Grundsätzlich unterscheidet man beim Training eines Modells zwischen überwachtem und unüberwachtem Lernen (*supervised* vs. *unsupervised learning*). Bei überwachtem Lernen wird ein Algorithmus mit annotierten Daten gefüttert, z.B. Bildern von Katzen und anderen Tieren, wobei jedes Bild mit dem Label "Katze" oder "Nicht-Katze" versehen ist. Ein Katzendetektor-Algorithmus hat beim Training Zugang zur "Wahrheit", also zur korrekten Antwort ("Katze" oder "Nicht-Katze"). Ziel des Trainings ist es, dass der Algorithmus über die Trainingsdaten hinaus zu *generalisieren* lernt, d.h. dass er nach dem Training auch unannotierten Input korrekt als Katze oder Nicht-Katze bestimmen kann. 

Unüberwachtes Lernen hingegen benötigt keinerlei menschlichen Input, abgesehen von unannotierten Trainingsdaten. Word2vec ist ein unüberwachter Algorithmus. Fürs Training schafft sich word2vec ganz einfach seine eigenen Trainingsdaten: Basierend auf den vorangehenden und folgenden Wörtern innerhalb eines Satzes versucht word2vec ein verdecktes Wort in der Mitte vorherzusagen: Bei "Der schwarze Hund _____ laut" wäre "bellt" z.B. eine gute Vorhersage. Die Idee hinter word2vec und Word Embeddings im Allgemeinen ist nun, dass der Algorithmus für ähnliche Wörter ähnliche Vektoren erlernt, denn ähnliche Wörter kommen in ähnlichen Kontexten (mit ähnlichen vorangehenden und folgenden Wörter) vor. "knurrt", das ebenfalls ein guter Lückenfüller wäre, sollte demnach einen ähnlichen Vektor wie "bellt" haben. Die Ähnlichkeit von Wörtern gemäß unseres Sprachmodells schauen wir uns unten im Detail an. Zuerst müssen wir das Modell nun aber trainieren.

---

Wir legen dazu ein paar Parameter fest, u.a.:

- `vector_size`, das die Anzahl an Features pro Vektor festlegt.
- `window` das festlegt, wie viele Wörter vor bzw. nach dem verdeckten Wort als Kontext beim Training berücksichtigt werden sollen.
- `min_count`, das festlegt, wie oft ein Wort im Datensatz mindestens vorkommen muss, um beim Training berücksichtigt zu werden.
- `epochs`, das festlegt, wie oft der Algorithmus den ganzen Datensatz analysieren soll.

Und los geht"s! Das Training nimmt ein paar Sekunden in Anspruch. Es ist fertig, wenn das Sternchen links neben der Zelle unten durch eine Zahl ersetzt wird.

In [7]:
model = gensim.models.Word2Vec(preprocessed_sentences, vector_size=200, window=6, min_count=3) 
model.train(sentences, total_examples=len(sentences), epochs=10) 

2023-06-07 20:05:22,730 : INFO : collecting all words and their counts
2023-06-07 20:05:22,732 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-06-07 20:05:22,807 : INFO : PROGRESS: at sentence #10000, processed 157242 words, keeping 32960 word types
2023-06-07 20:05:22,861 : INFO : PROGRESS: at sentence #20000, processed 307188 words, keeping 52920 word types
2023-06-07 20:05:22,913 : INFO : PROGRESS: at sentence #30000, processed 463034 words, keeping 69805 word types
2023-06-07 20:05:22,955 : INFO : PROGRESS: at sentence #40000, processed 613719 words, keeping 84454 word types
2023-06-07 20:05:23,016 : INFO : PROGRESS: at sentence #50000, processed 763044 words, keeping 97202 word types
2023-06-07 20:05:23,074 : INFO : PROGRESS: at sentence #60000, processed 915275 words, keeping 110022 word types
2023-06-07 20:05:23,111 : INFO : PROGRESS: at sentence #70000, processed 1070918 words, keeping 122585 word types
2023-06-07 20:05:23,153 : INFO : PROGRESS: 

(9657209, 15439430)

Nun ist das Modell fertig trainiert.

Schauen wir uns mal den Vektor des Worts "Universität" an.

In [8]:
search_term = "Universität"
print(model.wv[search_term]) #wv steht für word vector

[ 1.76908806e-01 -6.20428979e-01  8.04425418e-01  5.98221958e-01
  3.54229093e-01  5.86936712e-01  4.21542190e-02 -2.70387679e-01
 -8.12268853e-01  2.17557669e-01 -5.38275652e-02 -6.42810389e-02
 -6.98471427e-01 -1.42438188e-01 -1.74809113e-01 -6.62084669e-02
  7.37658739e-01  1.70509338e-01  1.81972817e-01 -6.62572622e-01
  1.02246296e+00 -2.77587742e-01  6.45766497e-01 -5.47019899e-01
 -6.13124013e-01  1.21455632e-01  5.43121621e-02 -1.27894491e-01
  1.28955245e-01 -6.39330447e-01 -1.07921518e-01  2.71578282e-02
  3.75380576e-01  5.06270647e-01  3.71149749e-01  5.35258591e-01
  1.58502474e-01  5.89708805e-01  2.85069108e-01 -1.82113066e-01
  7.29672194e-01 -4.83986050e-01  5.47285020e-01  7.11392105e-01
  4.93690223e-01 -1.01192132e-01  5.81915915e-01 -7.60676324e-01
  7.19942987e-01  8.66918623e-01 -5.15697658e-01 -2.63820022e-01
 -4.75525558e-01 -6.10494435e-01  1.15849838e-01 -1.29498827e+00
 -5.11104763e-01 -1.66428000e-01  3.17165852e-01 -6.09246969e-01
  1.87628239e-01 -3.39562

Spiel gerne mit anderen Wörter herum, indem Du sie bei `search_term` zwischen den Anführungszeichen einsetzt. Bei Wörtern, die das Modell nicht kennt (weil sie nicht (genügend oft) in unseren Daten vorkamen) erhältst Du einen `KeyError`.

## 4. Das Modell

Insgesamt sind diese Vektoren komplett nichtssagend für unser Sprachverständnis. Schauen wir aber, ob das Modell dennoch die Semantik von "Universität" auf seine eigene, vektorielle Weise einfangen konnte.

### 4.1. Ähnlichkeiten

Dies können wir etwa tun, in dem wir uns das ähnlichste Wort im Sprachmodell ausgeben lassen, also dasjenige Wort mit dem ähnlichsten Vektor:

In [10]:
search_term = "Universität"
most_similar = pd.DataFrame(model.wv.most_similar(search_term, topn=10), columns=["Word", "Similarity"], index=range(1,11))
most_similar.head(1)

,Word,Similarity
1,Hochschule,0.752539


Vermutlich kommt "Hochschule" als ähnlichstes Wort heraus. Faszinierend, oder? Setze gerne weitere Wörter bei `search_term` zwischen den Anführungszeichen ein!

⚠️ Achtung: Hier steht "vermutlich", da jedes Modell unterschiedlich ist, auch wenn die Trainingsdaten identisch waren. Dies liegt daran, dass unser Algorithmus nicht *deterministisch* ist. Ganz am Anfang des Trainings werden den Features der Vektoren nämlich zufällige Werte zugewiesen. Das Training des Algorithmus besteht dann darin, die Features von Runde zu Runde anzupassen, um bessere Vorhersagen zu erzielen (also das verdeckte Wort besser zu erraten).

Schauen wir uns die nächstähnlichen Wörter zu "Universität" ebenfalls an:

In [11]:
most_similar.head(10)

,Word,Similarity
1,Hochschule,0.752539
2,Uni,0.679624
3,Börse,0.673614
4,Spitze,0.672711
5,TU,0.670073
6,AG,0.663688
7,Kirchengemeinde,0.662947
8,Tabellenspitze,0.656888
9,Raumstation,0.655456
10,Infektiologie,0.654944


Spätestens hier sollten sich Unterschiede zwischen verschiedenen Modellen zeigen. Vermutlich sind auch nicht mehr alle Wörter für unser menschliches Sprachgefühl ähnlich zu "Universität". 

Wir könnten nun mit den Parametern oben experimentieren (z.B. mehr Epochen oder ein größeres/kleineres Kontextfenster), um bessere Resultate zu erzielen. Zielführender ist es jedoch, das Modell mit einem größeren Datensatz zu füttern. Die Quantität an Trainingsdaten ist absolut entscheidend für ein gutes Sprachmodell, wobei die Qualität der Trainingsdaten (d.h. z.B., ob sie ausgewogen und repräsentativ sind) auch nicht außer Acht gelassen werden sollte! 

Da wir an der Qualität kurzfristig nichts ändern können, verzehnfachen wir einfach mal die Quantität. Das Training mit 1 Million Sätze aus dem Wortschatz Leipzig würde relativ lange dauern, weswegen wir ganz einfach das bereits trainierte Modell "word2vec_1m.model" aus dem Ordner "model" in den Arbeitsspeicher laden:

In [16]:
model = gensim.models.Word2Vec.load("model/word2vec_1M.model")

2023-06-07 20:08:03,441 : INFO : loading Word2Vec object from model/word2vec_1M.model


TypeError: __randomstate_ctor() takes from 0 to 1 positional arguments but 2 were given

Schauen wir uns nun die zehn ähnlichsten Wörter zu "Universität" an:

In [17]:
search_term = "Universität"
most_similar = pd.DataFrame(model.wv.most_similar(search_term, topn=10), columns=["Word", "Similarity"], index=range(1,11))
most_similar.head(10)

,Word,Similarity
1,Hochschule,0.752539
2,Uni,0.679624
3,Börse,0.673614
4,Spitze,0.672711
5,TU,0.670073
6,AG,0.663688
7,Kirchengemeinde,0.662947
8,Tabellenspitze,0.656888
9,Raumstation,0.655456
10,Infektiologie,0.654944


Dies sieht schon viel besser aus! 

Wenn wir nicht bloß an den ähnlichsten Wörtern zu einem bestimmten Wort interessiert sind, sondern daran, wie ähnlich bestimmte Wortpaare zueinander sind, können wir folgendermaßen vorgehen:

In [18]:
pairs = [
    ("Auto", "Fahrzeug"),   
    ("Auto", "Fahrrad"),   
    ("Auto", "Flugzeug"), 
    ("Auto", "Haferflocken"), 
    ("Auto", "Zahnbürste")]

for w1, w2 in pairs:
    print(f"{w1} und {w2:12} sind sich zu {model.wv.similarity(w1, w2)*100:.2f}% ähnlich.")

Auto und Fahrzeug     sind sich zu 86.07% ähnlich.
Auto und Fahrrad      sind sich zu 76.07% ähnlich.
Auto und Flugzeug     sind sich zu 61.23% ähnlich.
Auto und Haferflocken sind sich zu 28.95% ähnlich.


KeyError: "Key 'Zahnbürste' not present"

Auch interessant ist es, herauszufinden, welches Wort am wenigsten zu anderen gegebenen Wörtern passt:

In [19]:
print(model.wv.doesnt_match(["Berlin", "Hamburg", "Zürich", "Dresden"]))

Dresden


Experimentiere bei sämtlichen "Ähnlichkeitsmethoden" mit eigenen Begriffen herum, um herauszufinden, wie gut sich der Algorithmus Deinem Sprachverständnis nach Wortbedeutungen aneignen konnte.

Wir können uns den Vektorraum auch visualisieren lassen, zwar nicht in all seinen 200 Dimensionen (jedes Feature entspricht einer Koordinate in einer Dimension), da das für Menschen schlicht nicht vorstellbar ist, aber reduziert auf drei Dimensionen. Wir benutzen dazu den Embedding Projector von TensorFlow. [Hier]() findest Du eine Visualisierung des mit 1 Million Sätze trainierten Modells. 